# Chapter 8: The Naive Bayes Classifier

> (c) 2019 Galit Shmueli, Peter C. Bruce, Peter Gedeck 
>
> Code included in
>
> _Data Mining for Business Analytics: Concepts, Techniques, and Applications in Python_ (First Edition) 
> Galit Shmueli, Peter C. Bruce, Peter Gedeck, and Nitin R. Patel. 2019.

## Import required packages

In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt

!pip install dmba
from dmba import classificationSummary, gainsChart


     |████████████████████████████████| 11.8 MB 19.4 MB/s 
no display found. Using non-interactive Agg backend


## Import Dataset

* import dataset
* convert to categorical
* convert num to categorical by creating bins
* run NB model
* predict probabibilities using NB model

In [2]:
delays = pd.read_csv('FlightDelays.csv')

In [3]:
delays.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight Status
0,1455,OH,1455,JFK,184,01/01/2004,5935,BWI,0,4,1,N940CA,ontime
1,1640,DH,1640,JFK,213,01/01/2004,6155,DCA,0,4,1,N405FJ,ontime
2,1245,DH,1245,LGA,229,01/01/2004,7208,IAD,0,4,1,N695BR,ontime
3,1715,DH,1709,LGA,229,01/01/2004,7215,IAD,0,4,1,N662BR,ontime
4,1039,DH,1035,LGA,229,01/01/2004,7792,IAD,0,4,1,N698BR,ontime


In [4]:
delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CRS_DEP_TIME   2201 non-null   int64 
 1   CARRIER        2201 non-null   object
 2   DEP_TIME       2201 non-null   int64 
 3   DEST           2201 non-null   object
 4   DISTANCE       2201 non-null   int64 
 5   FL_DATE        2201 non-null   object
 6   FL_NUM         2201 non-null   int64 
 7   ORIGIN         2201 non-null   object
 8   Weather        2201 non-null   int64 
 9   DAY_WEEK       2201 non-null   int64 
 10  DAY_OF_MONTH   2201 non-null   int64 
 11  TAIL_NUM       2201 non-null   object
 12  Flight Status  2201 non-null   object
dtypes: int64(7), object(6)
memory usage: 223.7+ KB


In [5]:
delays.rename(columns={'Flight Status':'Flight_Status'},inplace=True)

In [6]:
delays.columns

Index(['CRS_DEP_TIME', 'CARRIER', 'DEP_TIME', 'DEST', 'DISTANCE', 'FL_DATE',
       'FL_NUM', 'ORIGIN', 'Weather', 'DAY_WEEK', 'DAY_OF_MONTH', 'TAIL_NUM',
       'Flight_Status'],
      dtype='object')

In [7]:
delays.DAY_WEEK = delays.DAY_WEEK.astype('category')
delays.CARRIER = delays.CARRIER.astype('category')
delays.ORIGIN = delays.ORIGIN.astype('category')
delays.DEST = delays.DEST.astype('category')

In [8]:
delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   CRS_DEP_TIME   2201 non-null   int64   
 1   CARRIER        2201 non-null   category
 2   DEP_TIME       2201 non-null   int64   
 3   DEST           2201 non-null   category
 4   DISTANCE       2201 non-null   int64   
 5   FL_DATE        2201 non-null   object  
 6   FL_NUM         2201 non-null   int64   
 7   ORIGIN         2201 non-null   category
 8   Weather        2201 non-null   int64   
 9   DAY_WEEK       2201 non-null   category
 10  DAY_OF_MONTH   2201 non-null   int64   
 11  TAIL_NUM       2201 non-null   object  
 12  Flight_Status  2201 non-null   object  
dtypes: category(4), int64(6), object(3)
memory usage: 164.4+ KB


In [9]:
delays.CRS_DEP_TIME = [round(t/100) for t in delays.CRS_DEP_TIME]
delays.head()

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status
0,15,OH,1455,JFK,184,01/01/2004,5935,BWI,0,4,1,N940CA,ontime
1,16,DH,1640,JFK,213,01/01/2004,6155,DCA,0,4,1,N405FJ,ontime
2,12,DH,1245,LGA,229,01/01/2004,7208,IAD,0,4,1,N695BR,ontime
3,17,DH,1709,LGA,229,01/01/2004,7215,IAD,0,4,1,N662BR,ontime
4,10,DH,1035,LGA,229,01/01/2004,7792,IAD,0,4,1,N698BR,ontime


In [10]:
delays.CRS_DEP_TIME = delays.CRS_DEP_TIME.astype('category')

In [11]:
outcome = 'Flight_Status'
predictors = ['DAY_WEEK','CARRIER','ORIGIN','DEST','CRS_DEP_TIME']

In [12]:
X = pd.get_dummies(delays[predictors])
y = (delays[outcome] == 'delayed').astype(int)
classes = ['ontime','delayed']

In [13]:
X.head()

,DAY_WEEK_1,DAY_WEEK_2,DAY_WEEK_3,DAY_WEEK_4,DAY_WEEK_5,DAY_WEEK_6,DAY_WEEK_7,CARRIER_CO,CARRIER_DH,CARRIER_DL,CARRIER_MQ,CARRIER_OH,CARRIER_RU,CARRIER_UA,CARRIER_US,ORIGIN_BWI,ORIGIN_DCA,ORIGIN_IAD,DEST_EWR,DEST_JFK,DEST_LGA,CRS_DEP_TIME_6,CRS_DEP_TIME_7,CRS_DEP_TIME_8,CRS_DEP_TIME_9,CRS_DEP_TIME_10,CRS_DEP_TIME_11,CRS_DEP_TIME_12,CRS_DEP_TIME_13,CRS_DEP_TIME_14,CRS_DEP_TIME_15,CRS_DEP_TIME_16,CRS_DEP_TIME_17,CRS_DEP_TIME_18,CRS_DEP_TIME_19,CRS_DEP_TIME_20,CRS_DEP_TIME_21
0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [14]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Flight_Status, dtype: int64

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=.4,random_state=1)

delays_nb = MultinomialNB(alpha=.1)
delays_nb.fit(X_train,y_train)

pred_prob_train = delays_nb.predict_proba(X_train)
pred_prob_valid = delays_nb.predict_proba(X_valid)

pd.DataFrame(pred_prob_train)

,0,1
0,0.538788,0.461212
1,0.969970,0.030030
2,0.983705,0.016295
3,0.969531,0.030469
4,0.905152,0.094848
...,...,...
1315,0.824814,0.175186
1316,0.648865,0.351135
1317,0.939830,0.060170
1318,0.540771,0.459229


In [16]:
y_train_pred = delays_nb.predict(X_train)
y_valid_pred = delays_nb.predict(X_valid)

In [17]:
pd.DataFrame(y_valid_pred)

,0
0,0
1,0
2,0
3,0
4,0
...,...
876,0
877,0
878,0
879,0


In [18]:
X_train.head()

,DAY_WEEK_1,DAY_WEEK_2,DAY_WEEK_3,DAY_WEEK_4,DAY_WEEK_5,DAY_WEEK_6,DAY_WEEK_7,CARRIER_CO,CARRIER_DH,CARRIER_DL,CARRIER_MQ,CARRIER_OH,CARRIER_RU,CARRIER_UA,CARRIER_US,ORIGIN_BWI,ORIGIN_DCA,ORIGIN_IAD,DEST_EWR,DEST_JFK,DEST_LGA,CRS_DEP_TIME_6,CRS_DEP_TIME_7,CRS_DEP_TIME_8,CRS_DEP_TIME_9,CRS_DEP_TIME_10,CRS_DEP_TIME_11,CRS_DEP_TIME_12,CRS_DEP_TIME_13,CRS_DEP_TIME_14,CRS_DEP_TIME_15,CRS_DEP_TIME_16,CRS_DEP_TIME_17,CRS_DEP_TIME_18,CRS_DEP_TIME_19,CRS_DEP_TIME_20,CRS_DEP_TIME_21
1215,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1476,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1897,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
83,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1172,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [19]:
index = X_train.index
indices = index.to_list()
# indices
delays.iloc[indices]

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status
1215,21,DH,2110,LGA,229,1/18/2004,7684,IAD,0,7,18,N632BR,ontime
1476,7,US,659,LGA,214,1/21/2004,2160,DCA,0,3,21,N713UW,ontime
1897,9,US,858,LGA,214,1/27/2004,2164,DCA,0,2,27,N733UW,ontime
83,13,US,1258,LGA,214,01/02/2004,2172,DCA,0,5,2,N736UW,ontime
1172,15,DL,1509,JFK,213,1/17/2004,746,DCA,0,6,17,N909DL,delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,18,MQ,1825,JFK,213,1/14/2004,4784,DCA,0,3,14,N728MQ,ontime
905,7,RU,653,EWR,169,1/13/2004,2703,BWI,0,2,13,N12530,ontime
1096,7,DL,739,LGA,214,1/16/2004,1742,DCA,0,5,16,N224DA,ontime
235,16,DH,1700,JFK,228,01/05/2004,7810,IAD,0,1,5,N331UE,ontime


In [20]:
y_train_pred = pd.DataFrame(y_train_pred)
y_train_pred.rename(columns={0:'delayed'},inplace=True)
y_train_pred.head()

,delayed
0,0
1,0
2,0
3,0
4,0


In [21]:
pred_prob_train = pd.DataFrame(pred_prob_train)
pred_prob_train.rename(columns={0:'ontime_prob',1:'delay_prob'},inplace=True)
pred_prob_train

,ontime_prob,delay_prob
0,0.538788,0.461212
1,0.969970,0.030030
2,0.983705,0.016295
3,0.969531,0.030469
4,0.905152,0.094848
...,...,...
1315,0.824814,0.175186
1316,0.648865,0.351135
1317,0.939830,0.060170
1318,0.540771,0.459229


In [22]:
train_predictions = pd.concat([delays.iloc[indices].reset_index(),pred_prob_train,y_train_pred],axis=1)
train_predictions.head()

,index,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status,ontime_prob,delay_prob,delayed
0,1215,21,DH,2110,LGA,229,1/18/2004,7684,IAD,0,7,18,N632BR,ontime,0.538788,0.461212,0
1,1476,7,US,659,LGA,214,1/21/2004,2160,DCA,0,3,21,N713UW,ontime,0.969970,0.030030,0
2,1897,9,US,858,LGA,214,1/27/2004,2164,DCA,0,2,27,N733UW,ontime,0.983705,0.016295,0
3,83,13,US,1258,LGA,214,01/02/2004,2172,DCA,0,5,2,N736UW,ontime,0.969531,0.030469,0
4,1172,15,DL,1509,JFK,213,1/17/2004,746,DCA,0,6,17,N909DL,delayed,0.905152,0.094848,0


In [23]:
index = X_valid.index
indices = index.to_list()
delays.iloc[indices]

,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status
1276,16,DH,1641,JFK,228,1/19/2004,7810,IAD,0,1,19,N327UE,ontime
1446,6,DL,630,LGA,214,1/21/2004,1740,DCA,0,3,21,N242DL,ontime
335,6,MQ,558,JFK,213,01/06/2004,4760,DCA,0,2,6,N739MQ,ontime
1458,18,DL,1828,LGA,214,1/21/2004,1764,DCA,0,3,21,N242DL,ontime
2038,18,US,1758,LGA,214,1/29/2004,2182,DCA,0,4,29,N704UW,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,14,DH,1439,EWR,213,01/07/2004,7307,IAD,0,3,7,N324UE,ontime
2063,13,RU,1259,EWR,213,1/29/2004,2692,IAD,0,4,29,N17507,ontime
159,17,RU,1738,EWR,199,01/03/2004,2097,DCA,0,6,3,N14998,ontime
2027,7,US,657,LGA,214,1/29/2004,2160,DCA,0,4,29,N751UW,delayed


In [24]:
y_valid_pred = pd.DataFrame(y_valid_pred)
y_valid_pred.rename(columns={0:'delayed'},inplace=True)
y_valid_pred.head()

,delayed
0,0
1,0
2,0
3,0
4,0


In [25]:
pred_prob_valid = pd.DataFrame(pred_prob_valid)
pred_prob_valid.rename(columns={0:'ontime_prob',1:'delay_prob'},inplace=True)
pred_prob_valid

,ontime_prob,delay_prob
0,0.540771,0.459229
1,0.970798,0.029202
2,0.780133,0.219867
3,0.970840,0.029160
4,0.982379,0.017621
...,...,...
876,0.741724,0.258276
877,0.787770,0.212230
878,0.779530,0.220470
879,0.973313,0.026687


In [26]:
valid_predictions = pd.concat([delays.iloc[indices].reset_index(),pred_prob_valid,y_valid_pred],axis=1)
valid_predictions.head()

,index,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status,ontime_prob,delay_prob,delayed
0,1276,16,DH,1641,JFK,228,1/19/2004,7810,IAD,0,1,19,N327UE,ontime,0.540771,0.459229,0
1,1446,6,DL,630,LGA,214,1/21/2004,1740,DCA,0,3,21,N242DL,ontime,0.970798,0.029202,0
2,335,6,MQ,558,JFK,213,01/06/2004,4760,DCA,0,2,6,N739MQ,ontime,0.780133,0.219867,0
3,1458,18,DL,1828,LGA,214,1/21/2004,1764,DCA,0,3,21,N242DL,ontime,0.970840,0.029160,0
4,2038,18,US,1758,LGA,214,1/29/2004,2182,DCA,0,4,29,N704UW,ontime,0.982379,0.017621,0


## Create a Probability table
First construct a frequency table and then convert it to the propability table

* create a frequency table for all the predictors
* create a proportion table for all the predictors
* calculate the probabiblity for the following scenarios:

1.  P(delayed | Carrier = DL, Day_Week = 7, Dep_Time = 10, Dest = LGA, Origin = DCA)
2. P(ontime | Carrier = DL, Day_Week = 7, Dep_Time = 10, Dest = LGA, Origin = DCA)
3. Total P(dealyed|....)
4. Total P(ontime|.....)



In [27]:
train_predictions.loc[(train_predictions.CARRIER == 'DL')&
                      (train_predictions.DAY_WEEK == 7)&
                      (train_predictions.CRS_DEP_TIME == 10)&
                      (train_predictions.DEST == 'LGA')&
                      (train_predictions.ORIGIN == 'DCA')]

,index,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status,ontime_prob,delay_prob,delayed
277,180,10,DL,1028,LGA,214,01/04/2004,1748,DCA,0,7,4,N221DL,ontime,0.941701,0.058299,0
1058,1748,10,DL,1026,LGA,214,1/25/2004,1748,DCA,0,7,25,N225DL,ontime,0.941701,0.058299,0
1119,706,10,DL,1028,LGA,214,01/11/2004,1748,DCA,0,7,11,N242DL,ontime,0.941701,0.058299,0


In [28]:
valid_predictions.loc[(valid_predictions.CARRIER == 'DL')&
                      (valid_predictions.DAY_WEEK == 7)&
                      (valid_predictions.CRS_DEP_TIME == 10)&
                      (valid_predictions.DEST == 'LGA')&
                      (valid_predictions.ORIGIN == 'DCA')]

,index,CRS_DEP_TIME,CARRIER,DEP_TIME,DEST,DISTANCE,FL_DATE,FL_NUM,ORIGIN,Weather,DAY_WEEK,DAY_OF_MONTH,TAIL_NUM,Flight_Status,ontime_prob,delay_prob,delayed
473,1225,10,DL,1029,LGA,214,1/18/2004,1748,DCA,0,7,18,N242DL,ontime,0.941701,0.058299,0


## Comparison of the manual calculated calculation vs the NB model prediction

In [39]:
train_df,valid_df = train_test_split(delays,test_size=.4,random_state=1)
for predictor in predictors:
  df = train_df[['Flight_Status',predictor]]
  freq_table = df.pivot_table(index='Flight_Status',columns = predictor ,aggfunc=len)
  prop_table = freq_table.apply(lambda x:x/sum(x),axis=1)  
  print(prop_table)
  print()
  print(freq_table)
  print()

DAY_WEEK              1         2         3  ...         5         6         7
Flight_Status                                ...                              
delayed        0.191571  0.149425  0.114943  ...  0.187739  0.068966  0.160920
ontime         0.124646  0.141643  0.144476  ...  0.169027  0.135977  0.104816

[2 rows x 7 columns]

DAY_WEEK         1    2    3    4    5    6    7
Flight_Status                                   
delayed         50   39   30   33   49   18   42
ontime         132  150  153  190  179  144  111

CARRIER              CO        DH        DL  ...        RU        UA        US
Flight_Status                                ...                              
delayed        0.057471  0.314176  0.095785  ...  0.218391  0.015326  0.068966
ontime         0.034939  0.229462  0.203966  ...  0.169027  0.016997  0.218130

[2 rows x 8 columns]

CARRIER        CO   DH   DL   MQ  OH   RU  UA   US
Flight_Status                                     
delayed        15   82 

## Confusion Matrix

* build a confusion matrix on the training set
* build a confusion matrix on the validation set

In [33]:
classificationSummary(y_valid,y_valid_pred,class_names=classes)

Confusion Matrix (Accuracy 0.7866)

        Prediction
 Actual  ontime delayed
 ontime     667      47
delayed     141      26
